In [1]:
import fractions as f
import itertools as it
import collections as col

# Hexany Generator

Following Narushima's book on Wilson's tuning systems.

In [2]:
def generate_hexany(factors: set) -> list:
    '''
    Generates the tone factors for all the (2, 4) combinations of elements in set factors.
    Arguments:
        factors
    Returns:
        combinations
    '''
    return list(it.combinations(factors, 2))

def octave_reduce(ratio: f.Fraction) -> f.Fraction:
    '''
    Octave reduce a just-intonation ratio.
    '''
    new_ratio = ratio
    while new_ratio < 1 or new_ratio >= 2:
        if new_ratio <= 1:
            new_ratio *= 2
        elif new_ratio >= 2:
            new_ratio /= 2
    return new_ratio

def generate_tones(factors: set) -> list:
    '''
    Generate octave-reduced tones from a set of factors
    '''
    nodes = generate_hexany(factors)
    max_fac = max(factors)
    tone_dict = {frozenset([node_A, node_B]) : octave_reduce(f.Fraction((node_A * node_B), max_fac)) for node_A, node_B in nodes}
    return tone_dict

def generate_harmonic_triads(factors: set) -> list:
    nodes = generate_hexany(factors)
    triad_list = []
    for comb in it.combinations(factors, 3):
        excluded = factors.difference(comb).pop()
        triad = [frozenset([node, excluded]) for node in comb]
        triad_list.append(triad)
    return triad_list

def generate_subharmonic_triads(factors: set) -> list:
    nodes = generate_hexany(factors)
    triad_list = []
    for comb in it.combinations(factors, 3):
        triad = [frozenset(filter(lambda x: x != node, comb)) for node in comb]
        triad_list.append(triad)
    return triad_list

def generate_tent(factors: set, triad_function) -> dict:
    tones = generate_tones(factors)
    triads = [[tones[node] for node in triad] for triad in triad_function(factors)]
    tent_dict = dict()
    for tone in tones.values():
        tent_list = []
        for triad in triads:
            if tone in triad:
                tent_list.append(tone)
                for chord_tone in triad:
                    if chord_tone != tone:
                        tent_list.append(chord_tone)
        tent_dict[tone] = tent_list

    ordered_tent_dict = col.OrderedDict(sorted(tent_dict.items()))
    return ordered_tent_dict

def generate_neighbors(factors: set, triad_function) -> dict:
    tones = generate_tones(factors)
    triads = [[tones[node] for node in triad] for triad in triad_function(factors)]
    tent_dict = dict()
    for tone in tones.values():
        tent_list = []
        for triad in triads:
            if tone in triad:
                for chord_tone in triad:
                    if chord_tone != tone:
                        tent_list.append(chord_tone)
        tent_dict[tone] = tent_list
    return tent_dict

def generate_chord_table(factors : set, triad_function) -> dict:
    tones = generate_tones(factors)
    triads = [set([tones[node] for node in triad]) for triad in triad_function(factors)]
    neighbors = generate_neighbors(factors, triad_function)
    chord_table = {}
    for note in neighbors.keys():
        chord_table[note] = {}
        for neighbor in neighbors[note]:
            for triad in triads:
                if triad.issuperset((note, neighbor)):
                    chord_table[note][neighbor] = triad.difference((note, neighbor)).pop()
    return chord_table

def lua_string_chord_table(factors: set) -> str:
    harm_dict = generate_chord_table(factors, generate_harmonic_triads)
    pprint = "CHORD_HARM = {}\n"
    for root in harm_dict.keys():
        first = True
        for sec_deg in harm_dict[root].keys():
            thi_deg = harm_dict[root][sec_deg]
            if first:
                pprint += f"CHORD_HARM[{root}] = " + "{}\n"
                first = False
            pprint += f"CHORD_HARM[{root}][{sec_deg}] = {thi_deg}\n"

    harm_dict = generate_chord_table(factors, generate_subharmonic_triads)
    pprint += "CHORD_SUBHARM = {}\n"
    for root in harm_dict.keys():
        first = True
        for sec_deg in harm_dict[root].keys():
            thi_deg = harm_dict[root][sec_deg]
            if first:
                pprint += f"CHORD_SUBHARM[{root}] = " + "{}\n"
                first = False
            pprint += f"CHORD_SUBHARM[{root}][{sec_deg}] = {thi_deg}\n"

    return pprint

def lua_string_tent(tent_dict: dict, harm: str, sequins = True) -> str:
    lua_string = f"{harm} = " + "{}\n"
    for tone in tent_dict.keys():
        if sequins:
            tone_string = f"{harm}[{tone}] = s" + "{"
        else:
            tone_string = f"{harm}[{tone}] = " + "{"
        first = True
        for chord_tone in tent_dict[tone]:
            if first:
                first = False
                tone_string += f"{chord_tone}"
            else:
                tone_string += f", {chord_tone}"
        tone_string += "}\n"
        lua_string += tone_string
    return lua_string

def explorer_string(factors: set) -> str:
    # Generate header
    lua_string = "-- Hexany "
    first = True
    for factor in factors:
        if first:
            first = False
            lua_string += f"{factor}"
        else:
            lua_string += f"-{factor}"
    # Generate SCALE table
    lua_string += "\nSCALE = {"
    tones = list(generate_tones(factors).values())
    tones.sort()
    first = True
    for tone in tones:
        if first:
            first = False
            lua_string += f"{tone}"
        else:
            lua_string += f", {tone}"
    lua_string += "}\n"
    # Generate HARM table
    harm_tent = generate_tent(factors, generate_harmonic_triads)
    lua_string += lua_string_tent(harm_tent, "HARM")
    lua_string += "\n"
    # Generate SUBHARM table
    harm_tent = generate_tent(factors, generate_subharmonic_triads)
    lua_string += lua_string_tent(harm_tent, "SUBHARM")
    lua_string += "\n"

    return lua_string

def wanderer_string(factors: set) -> str:
    # Generate header
    lua_string = "-- Hexany "
    first = True
    for factor in factors:
        if first:
            first = False
            lua_string += f"{factor}"
        else:
            lua_string += f"-{factor}"
    # Generate SCALE table
    lua_string += "\nMASTER_SCALE = {"
    tones = list(generate_tones(factors).values())
    tones.sort()
    first = True
    for tone in tones:
        if first:
            first = False
            lua_string += f"{tone}"
        else:
            lua_string += f", {tone}"
    lua_string += "}\n"
    # Generate HARM table
    harm_tent = generate_tent(factors, generate_harmonic_triads)
    lua_string += lua_string_tent(harm_tent, "HARM", sequins = False)
    lua_string += "\n"
    # Generate SUBHARM table
    harm_tent = generate_tent(factors, generate_subharmonic_triads)
    lua_string += lua_string_tent(harm_tent, "SUBHARM", sequins = False)
    lua_string += "\n"

    return lua_string

def print_scale(factors):
    tones = generate_tones(factors)
    tone_list = [tone for tone in tones.values()]
    tone_string = [str(tone) for tone in sorted(tone_list)]
    
    return "SCALE = {" + ', '.join(tone_string) + "}"

In [3]:
factors = set([1, 3, 5, 7])
tones = generate_tones(factors)
triads = [set([tones[node] for node in triad]) for triad in generate_harmonic_triads(factors)]
neighbors = generate_neighbors(factors, generate_harmonic_triads)

In [4]:
[(fac, tone) for fac, tone in sorted(tones.items(), key = lambda x: x[1])]

[(frozenset({1, 7}), Fraction(1, 1)),
 (frozenset({3, 5}), Fraction(15, 14)),
 (frozenset({5, 7}), Fraction(5, 4)),
 (frozenset({1, 5}), Fraction(10, 7)),
 (frozenset({3, 7}), Fraction(3, 2)),
 (frozenset({1, 3}), Fraction(12, 7))]

In [26]:
print(wanderer_string(factors))
print(lua_string_chord_table(factors))

-- Hexany 1-3-5-7
MASTER_SCALE = {1, 15/14, 5/4, 10/7, 3/2, 12/7}
HARM = {}
HARM[1] = {1, 3/2, 5/4, 1, 12/7, 10/7}
HARM[15/14] = {15/14, 10/7, 5/4, 15/14, 12/7, 3/2}
HARM[5/4] = {5/4, 1, 3/2, 5/4, 10/7, 15/14}
HARM[10/7] = {10/7, 15/14, 5/4, 10/7, 12/7, 1}
HARM[3/2] = {3/2, 1, 5/4, 3/2, 12/7, 15/14}
HARM[12/7] = {12/7, 15/14, 3/2, 12/7, 10/7, 1}

SUBHARM = {}
SUBHARM[1] = {1, 3/2, 12/7, 1, 5/4, 10/7}
SUBHARM[15/14] = {15/14, 10/7, 12/7, 15/14, 5/4, 3/2}
SUBHARM[5/4] = {5/4, 1, 10/7, 5/4, 3/2, 15/14}
SUBHARM[10/7] = {10/7, 15/14, 12/7, 10/7, 5/4, 1}
SUBHARM[3/2] = {3/2, 1, 12/7, 3/2, 5/4, 15/14}
SUBHARM[12/7] = {12/7, 15/14, 10/7, 12/7, 3/2, 1}


CHORD_HARM = {}
CHORD_HARM[12/7] = {}
CHORD_HARM[12/7][15/14] = 3/2
CHORD_HARM[12/7][3/2] = 15/14
CHORD_HARM[12/7][10/7] = 1
CHORD_HARM[12/7][1] = 10/7
CHORD_HARM[10/7] = {}
CHORD_HARM[10/7][15/14] = 5/4
CHORD_HARM[10/7][5/4] = 15/14
CHORD_HARM[10/7][12/7] = 1
CHORD_HARM[10/7][1] = 12/7
CHORD_HARM[1] = {}
CHORD_HARM[1][3/2] = 5/4
CHORD_HARM[1][

In [10]:
sorted([octave_reduce(f.Fraction(i/1)) for i in range(1, 14)])

[Fraction(1, 1),
 Fraction(1, 1),
 Fraction(1, 1),
 Fraction(1, 1),
 Fraction(9, 8),
 Fraction(5, 4),
 Fraction(5, 4),
 Fraction(11, 8),
 Fraction(3, 2),
 Fraction(3, 2),
 Fraction(3, 2),
 Fraction(13, 8),
 Fraction(7, 4)]

In [5]:
print(wanderer_string(factors))

-- Hexany 9-2-3-5
MASTER_SCALE = {1, 10/9, 5/4, 4/3, 3/2, 5/3}
HARM = {}
HARM[1] = {1, 4/3, 10/9, 1, 3/2, 5/4}
HARM[10/9] = {10/9, 5/4, 5/3, 10/9, 1, 4/3}
HARM[5/4] = {5/4, 10/9, 5/3, 5/4, 1, 3/2}
HARM[4/3] = {4/3, 3/2, 5/3, 4/3, 1, 10/9}
HARM[3/2] = {3/2, 4/3, 5/3, 3/2, 1, 5/4}
HARM[5/3] = {5/3, 5/4, 10/9, 5/3, 3/2, 4/3}

SUBHARM = {}
SUBHARM[1] = {1, 4/3, 3/2, 1, 10/9, 5/4}
SUBHARM[10/9] = {10/9, 5/4, 1, 10/9, 5/3, 4/3}
SUBHARM[5/4] = {5/4, 10/9, 1, 5/4, 5/3, 3/2}
SUBHARM[4/3] = {4/3, 3/2, 1, 4/3, 5/3, 10/9}
SUBHARM[3/2] = {3/2, 4/3, 1, 3/2, 5/3, 5/4}
SUBHARM[5/3] = {5/3, 5/4, 3/2, 5/3, 10/9, 4/3}




In [33]:
triads

[{Fraction(10, 9), Fraction(5, 4), Fraction(5, 3)},
 {Fraction(4, 3), Fraction(3, 2), Fraction(5, 3)},
 {Fraction(1, 1), Fraction(10, 9), Fraction(4, 3)},
 {Fraction(1, 1), Fraction(5, 4), Fraction(3, 2)}]

In [30]:
a.issuperset((1,2))

True

{Fraction(1, 1): [Fraction(4, 3),
  Fraction(10, 9),
  Fraction(3, 2),
  Fraction(5, 4)],
 Fraction(3, 2): [Fraction(4, 3),
  Fraction(5, 3),
  Fraction(1, 1),
  Fraction(5, 4)],
 Fraction(5, 4): [Fraction(10, 9),
  Fraction(5, 3),
  Fraction(1, 1),
  Fraction(3, 2)],
 Fraction(4, 3): [Fraction(3, 2),
  Fraction(5, 3),
  Fraction(1, 1),
  Fraction(10, 9)],
 Fraction(10, 9): [Fraction(5, 4),
  Fraction(5, 3),
  Fraction(1, 1),
  Fraction(4, 3)],
 Fraction(5, 3): [Fraction(5, 4),
  Fraction(10, 9),
  Fraction(3, 2),
  Fraction(4, 3)]}

In [17]:
a = [1,2,3]
b = a.copy()

In [7]:
factors = set([2, 3, 5, 9])
print(explorer_string(factors))

-- Hexany 9-2-3-5
SCALE = {1, 10/9, 5/4, 4/3, 3/2, 5/3}
HARM = {}
HARM[1] = s{1, 4/3, 10/9, 1, 3/2, 5/4}
HARM[10/9] = s{10/9, 5/4, 5/3, 10/9, 1, 4/3}
HARM[5/4] = s{5/4, 10/9, 5/3, 5/4, 1, 3/2}
HARM[4/3] = s{4/3, 3/2, 5/3, 4/3, 1, 10/9}
HARM[3/2] = s{3/2, 4/3, 5/3, 3/2, 1, 5/4}
HARM[5/3] = s{5/3, 5/4, 10/9, 5/3, 3/2, 4/3}

SUBHARM = {}
SUBHARM[1] = s{1, 4/3, 3/2, 1, 10/9, 5/4}
SUBHARM[10/9] = s{10/9, 5/4, 1, 10/9, 5/3, 4/3}
SUBHARM[5/4] = s{5/4, 10/9, 1, 5/4, 5/3, 3/2}
SUBHARM[4/3] = s{4/3, 3/2, 1, 4/3, 5/3, 10/9}
SUBHARM[3/2] = s{3/2, 4/3, 1, 3/2, 5/3, 5/4}
SUBHARM[5/3] = s{5/3, 5/4, 3/2, 5/3, 10/9, 4/3}


